In [1]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm
import pandas as pd

print(f"Running on PyMC3 v{pm.__version__}")

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
Running on PyMC3 v3.11.2


In [2]:
# Initialize random number generator
RANDOM_SEED = 8927
np.random.seed(RANDOM_SEED)
az.style.use("arviz-darkgrid")

In [3]:
df = pd.read_csv("C:\\Users\\Scott\\Baysesian_Project\\train.csv")
df.head()

,Unnamed: 0,Country,Year,Status,Life.expectancy,Adult.Mortality,infant.deaths,Alcohol,percentage.expenditure,Hepatitis.B,...,Polio,Total.expenditure,Diphtheria,HIV.AIDS,GDP,Population,thinness..1.19.years,thinness.5.9.years,Income.composition.of.resources,Schooling
0,12,Afghanistan,2004,Developing,57.0,293,87,0.02,15.296066,67.0,...,5,8.79,5,0.1,219.141353,24118979.0,19.5,19.7,0.381,6.8
1,13,Afghanistan,2003,Developing,56.7,295,87,0.01,11.089053,65.0,...,41,8.82,41,0.1,198.728544,2364851.0,19.7,19.9,0.373,6.5
2,14,Afghanistan,2002,Developing,56.2,3,88,0.01,16.887351,64.0,...,36,7.76,36,0.1,187.845950,21979923.0,19.9,2.2,0.341,6.2
3,15,Afghanistan,2001,Developing,55.3,316,88,0.01,10.574728,63.0,...,35,7.80,33,0.1,117.496980,2966463.0,2.1,2.4,0.340,5.9
4,16,Afghanistan,2000,Developing,54.8,321,88,0.01,10.424960,62.0,...,24,8.20,24,0.1,114.560000,293756.0,2.3,2.5,0.338,5.5


In [21]:
basic_model = pm.Model()

with basic_model:

    # Priors for unknown model parameters
    alpha = pm.Normal("alpha", mu=0, sigma=10)
    beta = pm.Normal("beta", mu=0, sigma=10, shape=2)
    sigma = pm.HalfNormal("sigma", sigma=1)

    # Expected value of outcome
    mu = alpha + beta[0] * np.sqrt(df['MA'].values)

    # Likelihood (sampling distribution) of observations
    Y_obs = pm.Normal("Y_obs", mu=mu, sigma=sigma, observed=df['Life.expectancy'].values)
    
    trace = pm.sample(500, return_inferencedata=False)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, beta, alpha]


Sampling 4 chains for 1_000 tune and 500 draw iterations (4_000 + 2_000 draws total) took 17 seconds.
The acceptance probability does not match the target. It is 0.9090007488741193, but should be close to 0.8. Try to increase the number of tuning steps.


In [23]:
with basic_model:
    display(az.summary(trace, round_to=2))

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,-17.64,0.38,-18.32,-16.93,0.01,0.01,684.32,899.31,1.0
beta[0],17.87,0.15,17.58,18.14,0.01,0.00,655.59,931.09,1.0
beta[1],0.16,9.41,-19.16,16.25,0.27,0.21,1208.45,1278.63,1.0
sigma,1.56,0.05,1.47,1.65,0.00,0.00,1554.40,1362.28,1.0
